# KoBERT 를 이용한 네이버 영화리뷰 분류

# 데이터 로드 및 정제

In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [2]:
# 네이버 영화 리뷰 데이터 학습을 위해 훈련 데이터와 테스트 데이터를 다운로드합니다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                           filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                           filename="ratings_test.txt")


('ratings_test.txt', <http.client.HTTPMessage at 0x7a5a89d5b710>)

In [3]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

len(train_data), len(test_data)

(150000, 50000)

In [4]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [7]:
train_data['document'].value_counts().head()  # 중복된 데이터들 많다. 이를 제거해야 한다

,count
document,
굿,181
good,92
최고,85
쓰레기,79
별로,66


In [8]:
train_data.loc[train_data['document'].isna()]  # 결측치 있다.  이 또한 제거해야 한다

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [9]:
# 중복데이터와 결측치 제거
train_data.drop_duplicates(subset=['document'], inplace=True) # document 컬럼에서 중복된 내용 제거
train_data.dropna(how='any', inplace=True)  # NaN 이 존재하는 행 제거

len(train_data)

146182

In [10]:
# 테스트 데이터는 결측치만 제거
test_data.dropna(how='any', inplace=True)

len(test_data)

49997

# BERT 의 입력

In [11]:
# BERT 의 입력은 세가지를 준비해야 한다
#  1. 정수인코딩
#  2. 세그먼트 인코딩  (문장구분)
#  3. 어텐션 마스크 (단어토큰, 패딩토큰 구분)

In [12]:
# BERT 한국어 사전학습된 토크나이저
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [13]:
tokenizer.tokenize("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역")

['보', '##는', '내내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']

In [14]:
encoded = tokenizer.encode("보는 내내 그대로 들어맞는 예측 카리스마 없는 악역")
encoded

[2, 1160, 2259, 6404, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]

In [15]:
# 2 -> [CLS]
# 3 -> [SEP]

In [16]:
tokenizer.decode(encoded)

'[CLS] 보는 내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [17]:
tokenizer.cls_token, tokenizer.cls_token_id

('[CLS]', 2)

In [18]:
tokenizer.sep_token, tokenizer.sep_token_id

('[SEP]', 3)

In [19]:
tokenizer.pad_token, tokenizer.pad_token_id

('[PAD]', 0)

In [20]:
# encode() : 정수인코딩 + 패딩 동시에 가능
#   max_length= : 최대 인코딩 길이
#   padding='max_length' : 최대길이까지 패딩
#   truncation=True

## 정수 인코딩 + 패딩

In [21]:
max_seq_len = 128
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고 싶은 영화",
                 padding='max_length', max_length=max_seq_len, truncation=True)
print(encoded_result)
print('길이:', len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 1335, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이: 128


## 세그먼트 인코딩

In [22]:
# 어짜피 '하나의 텍스트' 이미 0으로 채워서 준비한다
print([0] * max_seq_len)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 어텐션 마스크 인코딩

In [23]:
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고 싶은 영화"))  # 패딩 없는 텍스트를 인코딩

print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
# 입력된 전체 데이터에 대해 위 과정을 진행하는 함수를 만들자.

def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
  # input_ids: 워드 임베딩을 위한 문장의 정수인코딩
  # attention_masks: 어텐션 마스크 인코딩
  # token_type_ids: 세그먼트 인코딩
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    input_id = tokenizer.encode(example, padding='max_length', max_length=max_seq_len, truncation=True)
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  # 최종 리턴은 array
  input_ids = np.array(input_ids, dtype=int) # 정수인코딩 + 패딩
  attention_masks = np.array(attention_masks, dtype=int)  # 어텐션 마스크
  token_type_ids = np.array(token_type_ids, dtype=int)  # 세그먼트 인코딩

  data_labels = np.asarray(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels

In [25]:
# 훈련 데이터에 대해서 진행.
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'],
                                                max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 146182/146182 [00:42<00:00, 3411.73it/s]


In [26]:
# 테스트 데이터에 대해서 진행.
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 49997/49997 [00:14<00:00, 3491.34it/s]


In [27]:
# 훈련 데이터의 첫번째 샘플에 대해서 출력해보겠습니다.

# return (input_ids, attention_masks, token_type_ids), data_labels
#             0            1                2

input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩:', input_id)
print('어텐션 마스크:', attention_mask)
print('세그먼트 인코딩:', token_type_id)
print('각 인코딩 의 길이:', len(input_id))
print('정수 인코딩 복원:', tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩: [   2 1376  831 2604   18   18 4229 9801 2075 2203 2182 4243    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어텐션 마스크: [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트 인코딩: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

# BERT 의 출력

In [28]:
# 한국어 BERT 사전학습 모델
model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

In [29]:
# BERT 의 출력을 outputs 라는 변수에 저장.
max_seq_len = 128  # 입력문장의 길이는 128

input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])


In [30]:
# outputs 에는 두개의 출력이 존재

# 문장의 길이 만큼의 출력
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


In [31]:
print(outputs[1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


In [ ]:
# outputs[0] 은 (batch size, 128, 768)
#     문장의 길이 개수만큼의 출력.  Many-to-Many 태스크의 경우  outputs[0] 을 사용

# outputs[1] 은 (batch size, 768)
#     [CLS] 토큰 위치의 출력. Many-to_One 태스크의 경우 outputs[1] 을 사용.
#     지금과 같은 영화리뷰 분류 문제는 이에 해당

# BERT 를 이용한 Many-to-One 모델 만들기

In [32]:
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(1, # 이진분류 문제
                          kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), # 가중치 초기화 (평균0, 표준편차 0.02)
                          activation='sigmoid',
                          name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids,
              attention_mask=attention_mask, token_type_ids=token_type_ids)

    cls_token = outputs[1]  # Many-to-one task
    prediction = self.classifier(cls_token)

    return prediction






In [33]:
model = TFBertForSequenceClassification("klue/bert-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [34]:
model.fit(train_X, train_y, epochs=2, batch_size=64, validation_split=0.2)

Epoch 1/2
1828/1828 [==============================] - 2932s 2s/step - loss: 0.2825 - accuracy: 0.8809 - val_loss: 0.2379 - val_accuracy: 0.9046
Epoch 2/2
1828/1828 [==============================] - 2884s 2s/step - loss: 0.1862 - accuracy: 0.9268 - val_loss: 0.2398 - val_accuracy: 0.9072


In [35]:
# 평가
results = model.evaluate(test_X, test_y, batch_size=1024)
print('test losss, test acc: ', results)

49/49 [==============================] - 401s 8s/step - loss: 0.2489 - accuracy: 0.9017
test losss, test acc:  [0.24894863367080688, 0.9016541242599487]


# 예측

In [36]:
def sentiment_predict(new_sentence):

  # 세가지 입력 데이터 준비

  # 1.정수인코딩 + 패딩
  input_id = tokenizer.encode(new_sentence, padding='max_length', max_length=max_seq_len,
                   truncation=True)

  # 2.어텐션 마스크
  padding_count = input_id.count(tokenizer.pad_token_id)
  attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

  # 3. 세그먼트 인코딩
  token_type_id = [0] * max_seq_len

  # 위 입력 데이터를 numpy 로 변환
  input_ids = np.array([input_id])
  attention_masks = np.array([attention_mask])
  token_type_ids = np.array([token_type_id])

  encoded_input = [input_ids, attention_masks, token_type_ids]  # 입력 시퀀스 준비
  score = model.predict(encoded_input)[0][0]  # 첫번째 batch 의 출력값 0.0 ~ 1.0

  if score > 0.5:
    print(f'{score * 100:.2f}% 확률로 긍정 리뷰입니다')
  else:
    print(f'{(1-score) * 100:.2f}% 확률로 부정 리뷰입니다')




In [37]:
input_sentiments = [
    '보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에 ',
    "스토리는 확실히 실망이였지만 배우들 연기력이 대박이였다 특히 이제훈 연기 정말 ... 이 배우들로 이렇게밖에 만들지 못한 영화는 아쉽지만 배우들 연기력과 사운드는 정말 빛났던 영화. 기대하고 극장에서 보면 많이 실망했겠지만 평점보고 기대없이 집에서 편하게 보면 괜찮아요. 이제훈님 연기력은 최고인 것 같습니다",
    "남친이 이 영화를 보고 헤어지자고한 영화. 자유롭게 살고 싶다고 한다. 내가 무슨 나비를 잡은 덫마냥 나에겐 다시 보고싶지 않은 영화.",
    "이 영화 존잼입니다 대박",
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '이 영화 핵노잼 ㅠㅠ',
    '감독 뭐하는 놈이냐?',
    '와 개쩐다 정말 세계관 최강자들의 영화다',
]

In [38]:
for sentiment in input_sentiments:
  print(sentiment)
  sentiment_predict(sentiment)
  print('🟦' * 20)

보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에 
1/1 [==============================] - 3s 3s/step
99.14% 확률로 부정 리뷰입니다
🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦
스토리는 확실히 실망이였지만 배우들 연기력이 대박이였다 특히 이제훈 연기 정말 ... 이 배우들로 이렇게밖에 만들지 못한 영화는 아쉽지만 배우들 연기력과 사운드는 정말 빛났던 영화. 기대하고 극장에서 보면 많이 실망했겠지만 평점보고 기대없이 집에서 편하게 보면 괜찮아요. 이제훈님 연기력은 최고인 것 같습니다
1/1 [==============================] - 0s 52ms/step
98.51% 확률로 긍정 리뷰입니다
🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦
남친이 이 영화를 보고 헤어지자고한 영화. 자유롭게 살고 싶다고 한다. 내가 무슨 나비를 잡은 덫마냥 나에겐 다시 보고싶지 않은 영화.
1/1 [==============================] - 0s 49ms/step
62.60% 확률로 부정 리뷰입니다
🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦
이 영화 존잼입니다 대박
1/1 [==============================] - 0s 49ms/step
97.95% 확률로 긍정 리뷰입니다
🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦
이 영화 개꿀잼 ㅋㅋㅋ
1/1 [==============================] - 0s 48ms/step
97.66% 확률로 긍정 리뷰입니다
🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦
이 영화 핵노잼 ㅠㅠ
1/1 [==============================] - 0s 50ms/step
97.91% 확률로 부정 리뷰입니다
🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦
감독 뭐하는 놈이냐?
1/1 [==============================] - 0s 53ms/step
99.32% 확률로 부정 리뷰입니다
🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦🟦
와 개